# <center> Этап 1 | Обучающая выборка | Ранжирование <center>

## 1. Построение обучающей выборки
Пусть 
$$
\begin{cases}
    E = \sigma_1 (x + D)\\
    S_x = \sigma_2 (x + D)
\end{cases}
-D < x < 0
$$
$$
\begin{cases}
    G = (x + D_0)^2\\
    S_{\tau} = cos(2\pi\tau) + 1
\end{cases}
0 < \tau < 1
$$
Возьмем в качестве $v_i$
$$x_{yng} = A_{yng} + B_{yng}cos(2\pi\tau)\ \ \ \ \ \ x_{old} = A_{old} + B_{old}cos(2\pi\tau)$$

Ключевые параметры: 
$$M_1(v_i) = \int_0^1 E(x_{yngi}(\tau))d\tau\ \ \ \ \ \ M_2(v_i) = \int_0^1 S_{\tau}(\tau)S_x(x_{yngi}(\tau))d\tau\ \ \ \ \ \ M_3(v_i) = -\int_0^1 (x'_{yngi}(\tau))d\tau\ \ \ \ \ \ M_4(v_i) = -\int_0^1 G(x_{yngi}(\tau))d\tau$$





In [ ]:
import csv
import random as rn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rand_A_B(Nmax):
   A_ = []
   B_ = []
   for i in range(0, Nmax):
      A = round(rn.uniform(-D, 0), 3)
      B = round(rn.uniform(-min(A + D, -A), min(A + D, -A)), 3)
      A_.append(A)
      B_.append(B)
   return A_, B_

In [ ]:
def strategy(t, A, B):
   x_yng = []
   for i in range(0, Nmax):
      x_yng_ = A[i] + B[i]*np.cos(2*np.pi*t[i])
      x_yng_ = round(x_yng_, 3)
      x_yng.append(x_yng_)
   return x_yng

In [ ]:
Nmax = 20
D = 120
D0 = 70  
sigma1 = 0.25
sigma2 = 0.0003

AB = rand_A_B(20)
A = AB[0]
B = AB[1]
t = np.linspace(0, 1, Nmax)
x = strategy(t, A, B)
print(x)

plt.plot(t, x)
